In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.regression_models as RegressionModels
import core.assignment_models as AssignmentModels
import core
import keras
import core.regression_models.RegressionTransformer as RegressionTransformer

plt.rcParams["font.size"] = 14

PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config_HLF.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=4_000_000
)

X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)

2025-11-28 23:56:40.378966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764370600.401557 3562078 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764370600.408951 3562078 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764370600.427103 3562078 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764370600.427124 3562078 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764370600.427126 3562078 computation_placer.cc:177] computation placer alr

In [2]:
Transformer = RegressionTransformer.SimpleNeutrinoRegessor(data_config)

In [ ]:
Transformer.build_model(
    hidden_dim=256,
    num_heads=8,
    central_transformer_stack_size=4,
    neutrino_regression_layers=4,
    dropout_rate=0,
)
Transformer.compile_model(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4),
    loss={
        "regression": core.utils.MagnitudeDirectionLoss(),
        "assignment": core.utils.AssignmentLoss(),
    },
    metrics={
        "regression": core.utils.RegressionDeviation(),
        "assignment": core.utils.AssignmentAccuracy(),
    },
    loss_weights={
        "regression": 1.0,
        "assignment": 5.0,
    },
)


I0000 00:00:1764370700.633609 3562078 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [ ]:
Transformer.train_model(
    X_train,
    y_train,
    batch_size=1024,
    copy_data=True,
    epochs=50,
)

Epoch 1/50


I0000 00:00:1764370719.994562 3563842 service.cc:152] XLA service 0x7f7a380104d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764370719.994589 3563842 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-28 23:58:40.604964: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-28 23:58:41.321735: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1764370723.201242 3563842 cuda_dnn.cc:529] Loaded cuDNN version 91500


   2/2809 ━━━━━━━━━━━━━━━━━━━━ 3:55 84ms/step - assignment_accuracy: 0.1477 - assignment_loss: 1.6386 - loss: 120.3324 - regression_deviation: 46.7100 - regression_loss: 112.1394   

I0000 00:00:1764370741.272071 3563842 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2808/2809 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - assignment_accuracy: 0.3345 - assignment_loss: 1.2729 - loss: 8.4109 - regression_deviation: 4.0461 - regression_loss: 2.0464

...

2809/2809 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - assignment_accuracy: 0.3346 - assignment_loss: 1.2729 - loss: 8.4105 - regression_deviation: 4.0459 - regression_loss: 2.0461

2025-11-29 00:03:00.475155: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-29 00:03:11.000137: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2809/2809 ━━━━━━━━━━━━━━━━━━━━ 289s 90ms/step - assignment_accuracy: 0.3346 - assignment_loss: 1.2728 - loss: 8.4100 - regression_deviation: 4.0458 - regression_loss: 2.0459 - val_assignment_accuracy: 0.4604 - val_assignment_loss: 1.0212 - val_loss: 6.1695 - val_regression_deviation: 2.9234 - val_regression_loss: 1.0638
Epoch 2/50
2809/2809 ━━━━━━━━━━━━━━━━━━━━ 225s 80ms/step - assignment_accuracy: 0.4420 - assignment_loss: 1.0885 - loss: 6.5705 - regression_deviation: 3.4690 - regression_loss: 1.1278 - val_assignment_accuracy: 0.4797 - val_assignment_loss: 0.9974 - val_loss: 5.9583 - val_regression_deviation: 3.2113 - val_regression_loss: 0.9708
Epoch 3/50
2809/2809 ━━━━━━━━━━━━━━━━━━━━ 225s 80ms/step - assignment_accuracy: 0.4515 - assignment_loss: 1.0639 - loss: 6.3819 - regression_deviation: 3.5451 - regression_loss: 1.0625 - val_assignment_accuracy: 0.4765 - val_assignment_loss: 0.9945 - val_loss: 5.9097 - val_regression_deviation: 3.2490 - val_regression_loss: 0.9374
Epoch 4/50
2

KeyboardInterrupt: 

: 

In [ ]:
Transformer.save_model(MODEL_DIR + "model.keras")